# ROOT

## Setting
Import the Python module for ROOT, and specify the path to the BNet file containing the Boolean model logic to be analyzed, as well as the directory where the results will be saved.

In [1]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

#Add the path to the folder containing the modules of the ROOT framework to sys.path.
path_of_ROOT_framework = r"D:\new canalizing kernel\ROOT framework"

import sys, os
sys.path.append(path_of_ROOT_framework)

In [2]:
#import python modules of ROOT

from Model_read_using_pyboolnet import read_pyboolnet_file
from iATG_module import iATG

In [ ]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################


# Assign the path to the folder for saving the results and 
# the path to the .bnet file containing the logic of the model to the variables.

directory_for_save = r"D:\new canalizing kernel\ROOT framework\ROOT_application_examples\toy model"
model_address = os.path.join(directory_for_save, 'toy_model_Boolean_logics.bnet')

## Determine input configuration change to analyze
Specify the input configurations to be analyzed.<br>
The input configuration before the irreversible process occurs is referred to as the basal input configuration, and the one that induces the irreversible process is referred to as the transition input configuration.<br><br>
For each node included in the input configuration, create a dictionary where the node is the key and the Boolean value it has in the basal input configuration is the value. Store this dictionary in a variable named `IC_basal`.<br>
Similarly, create a dictionary for the transition input configuration and store it in the variable `IC_transition`.

If mutations are to be incorporated into the model, the corresponding mutation information should be provided.<br>
Mutations can be represented as the fixation of specific nodes in the network model to either the ON (1) or OFF (0) state.<br>
This information should be entered into the `mutated_node_state_map` dictionary, where the key is the name of the mutated node and the value is the fixed Boolean state assigned to that node.

In [4]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

IC_basal = {"Input":1}
IC_transition = {"Input":0}
mutated_node_state_map = {}

## Construct an iATG
After creating the iATG and dynamics objects, perform the computation to construct the iATG by setting the appropriate parameter values.

In [5]:
dynamics_of_model = read_pyboolnet_file(model_address)
iATG_of_model = iATG(dynamics_of_model, IC_basal, IC_transition, mutated_node_state_map)

### Set the parameter values
- Set the parameter values required to compute the model’s iATG:
    - `use_all_initials`: Set this to either `True` or `False`.
        - If `True`, the attractor landscape for each input configuration is computed using all possible initial states of the Boolean model.
        - If `False`, only a subset of possible initial states is used to estimate the attractor landscape.
    - `waiting_num`: A positive integer.
        - Used _only_ when `use_all_initials` is `False`.
        - The larger this value is, the more initial states are used for attractor landscape estimation, resulting in higher estimation accuracy.
    - `difference_threshold`: A positive real number less than 1.
        - Used _only_ when `use_all_initials` is `False`.
        - The smaller this value is, the more initial states are used for attractor landscape estimation, improving accuracy.
    - `verbose`: Set this to either True or False.
        - When `use_all_initials` is `False`, setting `verbose` to `True` will display how many initial states are being explored during attractor landscape estimation.

In [6]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

use_all_initials=True
waiting_num = 10000
difference_threshold = 0.0001
verbose=False

using these parameters, iATG of the model is constructed

In [7]:
iATG_of_model.calculate_attractor_landscapes_for_each_IC(use_all_initials,
                                                        waiting_num, 
                                                        difference_threshold,
                                                        verbose)
iATG_of_model.get_attractor_transitions_induced_by_IC_change_and_calculate_TPs()

Calculating attractor landscapes using all initial states.
Calculating attractor landscape on input configuration {'Input': 1} using all initial states.
0.0% of all initial states are detected.
used time:  0.0
20.01953125% of all initial states are detected.
used time:  0.09081077575683594
40.0390625% of all initial states are detected.
used time:  0.41781187057495117
60.009765625% of all initial states are detected.
used time:  0.6044669151306152
80.029296875% of all initial states are detected.
used time:  0.7420833110809326
time for Attractor landscape for IC {'Input': 1}:  0.9042761325836182
Calculating attractor landscape on input configuration {'Input': 0} using all initial states.
0.0% of all initial states are detected.
used time:  0.0
20.01953125% of all initial states are detected.
used time:  0.03513622283935547
40.0390625% of all initial states are detected.
used time:  0.129807710647583
60.009765625% of all initial states are detected.
used time:  0.23131227493286133
80.02

## Check the constructed iATG and select appropriate ITPs.
After inspecting the constructed iATG, examine the irreversible transition paths (ITPs) present in it.<br>By analyzing the phenotypes of the attractors that compose each ITP, select the ITPs that correspond to the cellular irreversible processes under investigation.

### constructed iATG
The cell below displays the constructed iATG.<br>
Attractors included in the attractor landscape under the basal input configuration (`IC_basal`) are labeled with attractor codes in the form of `('basal', i)`, where `i` is an arbitrary index number.<br>
Attractors under the transition input configuration (`IC_transition`) are labeled as `('transition', i)`, where `i` is also an arbitrary index.<br><br>
An arrow between attractor codes (e.g., `('basal', 0) → ('transition', 1)`) indicates that if the model state remains at the attractor on the left (in this case, `('basal', 0)`), a transition to the attractor on the right (in this case, `('transition', 1)`) can occur upon a change in input configuration.

In [8]:
print("iATG constructed")
for attractor_transition in iATG_of_model.attractor_transitions_induced_by_IC_change:
    print("{}->{}".format(attractor_transition[0],attractor_transition[1]))

iATG constructed
('basal', 0)->('transition', 1)
('basal', 1)->('transition', 0)
('basal', 2)->('transition', 0)
('basal', 3)->('transition', 0)
('basal', 4)->('transition', 0)
('basal', 5)->('transition', 0)
('transition', 0)->('basal', 5)
('transition', 1)->('basal', 0)


### Phenotype check of ITPs
Using the constructed iATG, irreversible transition paths (ITPs) present in the iATG can be identified.<br>
The ITPs that exist in this iATG are listed in the output of the following cell.



In [9]:
ITP_index_map = {}
ITP_index = 0
iATG_of_model.find_iCAs_and_calculate_iCA_sizes()
for iCA in iATG_of_model.iCAs:
    iCA.search_ITPs()
    for ITP_of_model in iCA.ITPs:
        ITP_index_map[ITP_index] = ITP_of_model
        print("ITP index {}: {}".format(ITP_index, ITP_of_model))
        ITP_index += 1

ITP index 0: ITP from ('basal', 1) to ('transition', 0) to ('basal', 5)
ITP index 1: ITP from ('basal', 2) to ('transition', 0) to ('basal', 5)
ITP index 2: ITP from ('basal', 3) to ('transition', 0) to ('basal', 5)
ITP index 3: ITP from ('basal', 4) to ('transition', 0) to ('basal', 5)


Among the given ITPs, those corresponding to the cellular irreversible process under investigation should be selected for analysis.<br>
To do this, the average state information of the attractors that make up each ITP is utilized.

In [10]:
import pandas as pd

ITP_info_for_index = []
for index_of_ITP, ITP_of_model in ITP_index_map.items():
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_basal_irrev)))
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_transition)))
    ITP_info_for_index.append((index_of_ITP, str(ITP_of_model.attr_basal_rev)))


index_of_df = pd.MultiIndex.from_tuples(ITP_info_for_index, names=["ITP index", "attractor code"])

is_point_attractor = []
node_names_input_first = list(IC_basal.keys())
for node_name in dynamics_of_model.get_node_names():
    if node_name not in node_names_input_first:
        node_names_input_first.append(node_name)
node_averagestates_map = {node_name:[] for node_name in node_names_input_first}
for index_of_ITP, attractor_code in ITP_info_for_index:
    attractor_code = eval(attractor_code)#str to tuple
    attractor_object = iATG_of_model.get_attractor_using_attr_tuple_form(attractor_code)
    average_state = attractor_object.get_average_state()
    is_point_attractor.append(attractor_object.is_point_attractor())
    for node_name, list_of_average_states in node_averagestates_map.items():
        average_state_of_node = average_state.get(node_name, None)
        list_of_average_states.append(average_state_of_node)

df_for_ITP_info = pd.DataFrame({"is point attractor":is_point_attractor, **node_averagestates_map}, 
                               index=index_of_df)

The `df_for_ITP_info` calculated in the cell above contains the average state information of attractors included in each ITP, corresponding to each ITP index.<br>
Among the rows with the same ITP index value, 
- the first row shows the average state of the attractor corresponding to the ITP's 'attractor basal irreversible', 
- the second row corresponds to the 'attractor transition', 
- and the third row corresponds to the 'attractor basal reversible'.

<br>The values of `df_for_ITP_info` are displayed in the output of the cell below.

In [11]:
print("The average state for each attractor in ITPs")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_for_ITP_info)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

The average state for each attractor in ITPs


is point attractor  Input  Pheno1  Pheno2  X01  \
ITP index attractor code                                                      
0         ('basal', 1)                     True    1.0     1.0    1.00  1.0   
          ('transition', 0)                True    0.0     0.0    0.00  0.0   
          ('basal', 5)                     True    1.0     0.0    0.00  0.0   
1         ('basal', 2)                     True    1.0     1.0    0.00  1.0   
          ('transition', 0)                True    0.0     0.0    0.00  0.0   
          ('basal', 5)                     True    1.0     0.0    0.00  0.0   
2         ('basal', 3)                    False    1.0     1.0    0.75  1.0   
          ('transition', 0)                True    0.0     0.0    0.00  0.0   
          ('basal', 5)                     True    1.0     0.0    0.00  0.0   
3         ('basal', 4)                    False    1.0     0.5    0.50  0.5   
          ('transition', 0)                True    0.0     0.0    0.00  0.0   
          ('basal', 5)                     True    1.0     0.0    0.00  0.0   

                              X02   X03   X04   X05   X06   X07   X08  X09  
ITP index attractor code                                                    
0         ('basal', 1)       1.00  1.00  0.00  0.00  1.00  1.00  1.00  0.0  
          ('transition', 0)  0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
          ('basal', 5)       0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
1         ('basal', 2)       0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.0  
          ('transition', 0)  0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
          ('basal', 5)       0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
2         ('basal', 3)       0.75  0.75  0.25  0.25  0.75  0.75  0.75  0.0  
          ('transition', 0)  0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
          ('basal', 5)       0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
3         ('basal', 4)       0.50  1.00  0.00  0.00  1.00  1.00  1.00  0.5  
          ('transition', 0)  0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0  
          ('basal', 5)       0.00  1.00  0.00  0.00  1.00  1.00  1.00  1.0

### Selection of appropriate ITPs
Use the average state information of the attractors constituting each ITP to select the ITPs corresponding to the cellular irreversible process under investigation.<br>
You can add the `ITP index` of the ITPs you want to analyze by using the `append` method on `ITP_indices_to_analyze` in the cell directly below.<br>
Irreversibility kernel and reversion control method searches will then be performed on the selected ITPs.

The criteria for selecting ITPs are left to the user.<br>If multiple ITPs exist, methods such as clustering can also be employed.

If you wish to examine all constituent states of an attractor (rather than its average state), you can use:
`iATG_of_model.get_attractor_using_attr_tuple_form(attractor_code).show_states()`<br>
This command will display the constituent states of the attractor corresponding to the given attractor_code (which is a tuple type).

In [12]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

ITP_indices_to_analyze = []
ITP_indices_to_analyze.append(1)

## Analysis of selected ITPs
For each selected ITP, the irreversibility kernel is analyzed. The irreversibility kernel consists of irreversibility motifs—formed by positive feedback loops—and their corresponding coherency conditions, which stabilize each motif.

Using the irreversibility kernels identified for each ITP, reversion control methods are subsequently explored.

### Setting feedback length for irreversibility kernel construction
In the cell below, set the value of `max_len_of_feedback_search`.<br>
This parameter determines the maximum length of positive feedback loops (in the expanded network) to be searched as candidates for the irreversibility kernel.

- If `max_len_of_feedback_search` is set to 0, the search is conducted without any length limit.

- If it is set to a positive integer, only positive feedback loops with a length less than or equal to that number will be considered.

When the network has many nodes and dense connectivity, an unrestricted search can result in excessive computational cost. Therefore, it is recommended to set this value based on the available computational resources and time constraints.

In [13]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

max_len_of_feedback_search = 0

### Analysis and visualization of the irreversibility kernel
Perform the irreversibility kernel analysis for each ITP.

In [14]:

for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    
    # initialize the irreversibility motifs and coherency conditions in this object.
    # when you run this code more than once, you may want to clear the previous results.
    ITP_object.irreversibility_motifs = []
    ITP_object.coherency_conditions = []
    
    ITP_object.find_irreversibility_kernel(max_len_of_feedback_search)

Store the results of the irreversibility kernel analysis in `df_irreversibility_kernel`.

In [15]:
ITP_indices = []
irreversibilty_motifs = []
coherency_conditions = []

for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    for i, irreversibility_motif in enumerate(ITP_object.irreversibility_motifs):
        corresponding_coherency_condition = ITP_object.coherency_conditions[i]

        ITP_indices.append(ITP_index)
        irreversibilty_motifs.append(irreversibility_motif)
        coherency_conditions.append(corresponding_coherency_condition)

df_irreversibility_kernel = pd.DataFrame({"irreversibility motif":irreversibilty_motifs,
                                           "coherency condition":coherency_conditions},
                                          index=ITP_indices)
df_irreversibility_kernel.index.name = 'ITP index'

Structure of `df_irreversibility_kernel` is as follows.

Each row in `df_irreversibility_kernel` represents one irreversibility motif and its corresponding coherency condition for the ITP identified by the `ITP index` in that row.<br>
Since a single irreversibility kernel can consist of multiple irreversibility motifs and their corresponding coherency conditions, multiple rows can share the same ITP index.

- The `irreversibility motif` column lists the nodes that make up the irreversibility motif. In the network model’s structure, the subnetwork formed by these nodes contains combinations of positive feedbacks that constitute the irreversibility motif.
- The `coherency condition` column specifies the condition under which the `irreversibility motif` in the same row remains stable in the 'attractor basal reversible'. This is expressed as a dictionary where each key represents a node, and its value indicates the Boolean state that the node must maintain to satisfy the coherency condition.

In [16]:
print("Irreversibiltiy kernel for each ITP")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_irreversibility_kernel)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Irreversibiltiy kernel for each ITP


,irreversibility motif,coherency condition
ITP index,,
1,"{X06, X08}",{'X02': 0}


If we calculate the frequency with which each node appears in the irreversibility kernels of the selected ITPs, we obtain the following results.

In [17]:
all_nodes_in_irreversibility_kernel = set()
num_of_motifs = 0
num_of_kernels = 0
#ITPindex_motifnum_map = {}
for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    #ITPindex_motifnum_map[ITP_index] = 0
    
    if ITP_object.irreversibility_motifs:
        num_of_kernels += 1
        num_of_motifs += len(ITP_object.irreversibility_motifs)
        #ITPindex_motifnum_map[ITP_index] += len(ITP_object.irreversibility_motifs)
    
    for i, irreversibility_motif in enumerate(ITP_object.irreversibility_motifs):
        corresponding_coherency_condition = ITP_object.coherency_conditions[i]

        all_nodes_in_irreversibility_kernel.update(irreversibility_motif)
        all_nodes_in_irreversibility_kernel.update(corresponding_coherency_condition)
all_nodes_in_irreversibility_kernel = list(all_nodes_in_irreversibility_kernel)
all_nodes_in_irreversibility_kernel.sort()

kernel_containment_frequency = []
motif_containment_frequency = []
condition_containment_frequency = []

#motifcontainmentfrequency_map = {}

for node_name in all_nodes_in_irreversibility_kernel:
    motif_contatinment = 0
    condition_contatinment = 0
    kernel_contatinment = 0
    #motifcontainmentfrequency_map[node_name] = {}
    for ITP_index in ITP_indices_to_analyze:
        ITP_object = ITP_index_map[ITP_index]
        kernel_contatiment_flag = False
        #motifcontainmentfrequency_map[node_name][ITP_index] = 0
        
        for i, irreversibility_motif in enumerate(ITP_object.irreversibility_motifs):
            corresponding_coherency_condition = ITP_object.coherency_conditions[i]

            if node_name in irreversibility_motif:
                motif_contatinment += 1
                #motifcontainmentfrequency_map[node_name][ITP_index] += 1
                kernel_contatiment_flag = True
            if node_name in corresponding_coherency_condition:
                condition_contatinment += 1
                kernel_contatiment_flag = True

        if kernel_contatiment_flag:
            kernel_contatinment += 1
        #motifcontainmentfrequency_map[node_name][ITP_index] /= ITPindex_motifnum_map[ITP_index]

    kernel_containment_frequency.append(kernel_contatinment/num_of_kernels)
    motif_containment_frequency.append(motif_contatinment/num_of_motifs)
    condition_containment_frequency.append(condition_contatinment/num_of_motifs)

df_containment_frequency = pd.DataFrame({"kernel containment frequency":kernel_containment_frequency,
                                           "motif containment frequency":motif_containment_frequency,
                                           "condition containment frequency":condition_containment_frequency},
                                          index=all_nodes_in_irreversibility_kernel)      


The table shown in the cell below presents three types of frequencies for each node:

- `kernel containment frequency` indicates how often a given node appears in the irreversibility kernels of the analyzed ITPs. A value of 1 means that the node is included in the irreversibility kernels of all ITPs considered. In such cases, the node may be part of the irreversibility motif, the coherency condition, or both.

- `motif containment frequency` reflects how frequently the node appears within all irreversibility motifs identified across the ITPs.

- `condition containment frequency` represents the frequency with which the node is involved in all coherency conditions derived from the irreversibility kernels of the ITPs.

In [18]:
print("node containment frequency in irreversibility kernel")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_containment_frequency)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

node containment frequency in irreversibility kernel


,kernel containment frequency,motif containment frequency,condition containment frequency
X02,1.0,0.0,1.0
X06,1.0,1.0,0.0
X08,1.0,1.0,0.0


## Control configurations using the irreversibility kernel
For each ITP, control configurations for 'resetting control' and 'reversibility-inducing control' are computed using the corresponding irreversibility kernel.<br>
Each control configuration consists of a subset of nodes from the irreversibility motif (referred to as control targets) and the assigned states for each control target.

The cells below compute control configurations in which the number of control target nodes is equal to `num_of_control_targets`.<br>
If the control targets consist of nodes 'N1', 'N2', ..., 'Ni', and the assigned states are x1, x2, ..., xi, respectively, then the control configuration is represented as a dictionary: {'N1': x1, 'N2': x2, ..., 'Ni': xi}.<br>
For each ITP under analysis, candidate control configurations (control candidates) are calculated and stored in ITPindex_controlconfigurations_map, with the ITP index as the key and the corresponding control configurations as the value.

ROOT utilizes these control candidates to propose the most appropriate ones for either resetting control or reversibility-inducing control.

In [19]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

num_of_control_targets = 1

In [20]:
import pandas as pd

ITPindex_controlconfigurations_map = {}
ITP_indices = []
control_candidates_for_each_ITP_index = []

for ITP_index in ITP_indices_to_analyze:
    ITP_object = ITP_index_map[ITP_index]
    control_candidates_for_the_ITP = ITP_object.get_control_configurations_having_n_control_targets(num_of_control_targets)
    ITPindex_controlconfigurations_map[ITP_index] = control_candidates_for_the_ITP
    for control_candidate_for_the_ITP in control_candidates_for_the_ITP:
        ITP_indices.append(ITP_index)
        control_candidates_for_each_ITP_index.append(control_candidate_for_the_ITP)

df_control_candidates = pd.DataFrame({"control candidate":control_candidates_for_each_ITP_index},
                                      index=ITP_indices)
df_control_candidates.index.name = 'ITP index'

print("control candidates per ITP")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_control_candidates)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

control candidates per ITP


,control candidate
ITP index,
1,{'X06': 0}
1,{'X08': 0}


ROOT proposes two approaches to utilize the derived control candidates for control. These two approaches are as follows:
- resetting control
- reversibility-inducing control

### Resetting control approach

The objective of the resetting control approach is as follows:
- When the model state resides in the attractor_basal_rev of the target ITP,
- Without altering the model’s inherent dynamics permanently,
- Guide the model state toward the corresponding attractor_basal_irrev.

The resetting control procedure is carried out in the following steps:
- First, fix the control targets to their corresponding assigned states.
- Then, allow the model state to converge to a new attractor under the dynamics with control applied.
- After convergence, release the fixation of the control targets.
- Finally, the model state will transition to one of the attractors that exist in the original (uncontrolled) dynamics.

Each control configuration is associated with a 'reversion probability', which indicates the likelihood of achieving the goal of resetting control.<br>
ROOT assumes that, among all control configurations with the same number of control targets, those with the highest reversion probabilities are most likely to be included among the control candidates.<br>
Based on this assumption, ROOT calculates the reversion probability for each control configuration among the control candidates and uses these values to identify the most suitable control configurations for resetting control.

The cell below displays a table that ranks, for each ITP, the control configurations in the control candidates by their reversion probabilities in descending order. For each ITP, the control configurations with rank 1 are those proposed by ROOT as the most suitable for resetting control.

In [21]:
def compute_competition_ranks(values):
    indexed = list(enumerate(values))
    sorted_indexed = sorted(indexed, key=lambda x: -x[1])

    ranks = [0] * len(values)
    rank = 1
    for i, (idx, val) in enumerate(sorted_indexed):
        if i > 0 and val != sorted_indexed[i - 1][1]:
            rank = i + 1
        ranks[idx] = rank

    return ranks

control_configurations_for_all_analyzed_ITPs = []
for control_configurations in ITPindex_controlconfigurations_map.values():
    for control_configuration in control_configurations:
        if control_configuration not in control_configurations_for_all_analyzed_ITPs:
            control_configurations_for_all_analyzed_ITPs.append(control_configuration)

index_of_df = []
control_configurations_in_df = []
control_configuration_is_for_the_ITP = []
reversion_probabilities_in_df = []
rank_in_the_ITP = []

for ITP_index, control_configurations_for_the_ITP in ITPindex_controlconfigurations_map.items():
    ITP_object = ITP_index_map[ITP_index]
    for control_configuration in control_configurations_for_all_analyzed_ITPs:
        index_of_df.append(ITP_index)
        control_configurations_in_df.append(control_configuration)
        if control_configuration in control_configurations_for_the_ITP:
            control_configuration_is_for_the_ITP.append(True)
        else:
            control_configuration_is_for_the_ITP.append(False)
        reversion_probability = ITP_object.get_the_reversion_probability_for_resetting_control(control_configuration)
        reversion_probabilities_in_df.append(reversion_probability)
rank_in_the_ITP = compute_competition_ranks(reversion_probabilities_in_df)

df_resetting_control = pd.DataFrame({"control configuration":control_configurations_in_df,
                                     "control configuration is candidate control for the ITP":control_configuration_is_for_the_ITP,
                                       "reversion probability":reversion_probabilities_in_df,
                                       "rank in the ITP":rank_in_the_ITP},
                                      index=index_of_df)
df_resetting_control.index.name = 'ITP index'
df_resetting_control = df_resetting_control.sort_values("rank in the ITP")

print("control configurations and their reversion probabilities per ITP")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_resetting_control)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

control configurations and their reversion probabilities per ITP


,control configuration,control configuration is candidate control for the ITP,reversion probability,rank in the ITP
ITP index,,,,
1,{'X06': 0},True,1.0,1
1,{'X08': 0},True,1.0,1


The following table shows the average reversion probabilities for each control configuration, the frequency with which each was selected as a candidate control across ITPs, and their rank based on the average reversion probability. When attempting to apply resetting control across multiple ITPs simultaneously, the control configurations ranked 1 are those recommended by ROOT as the most suitable.

In [ ]:
reversion_probabilities_averaged = []
ratios_of_being_candidate_control = []
average_ranks = []
for control_configuration in control_configurations_for_all_analyzed_ITPs:
    sub_df_for_the_control_configuration = df_resetting_control[df_resetting_control["control configuration"] == control_configuration]
    reversion_probability_average_of_the_control_configuration = sub_df_for_the_control_configuration["reversion probability"].mean()
    reversion_probabilities_averaged.append(reversion_probability_average_of_the_control_configuration)
    ratio_of_being_condidate_for_the_control_configuration = sub_df_for_the_control_configuration["control configuration is candidate control for the ITP"].mean()
    ratios_of_being_candidate_control.append(ratio_of_being_condidate_for_the_control_configuration)
    average_ranks = compute_competition_ranks(reversion_probabilities_averaged)

df_resetting_control_statistic = pd.DataFrame({"average of reversion probabilities":reversion_probabilities_averaged,
                                     "ratio of being candidate control":ratios_of_being_candidate_control,
                                     "average rank":average_ranks},
                                      index=control_configurations_for_all_analyzed_ITPs)
df_resetting_control_statistic.index.name = 'control configuration'
df_resetting_control_statistic = df_resetting_control_statistic.sort_values("average rank")

print("Summary of averaged reversion probabilities and ratio of being candidate control")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_resetting_control_statistic)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Summary of averaged reversion probabilities and ratio of being candidate control


,average of reversion probabilities,ratio of being candidate control,average rank
control configuration,,,
{'X06': 0},1.0,1.0,1.0
{'X08': 0},1.0,1.0,1.0


### reversibility-inducing control
The goal of reversibility-inducing control is as follows:
- When the model state resides in the `attractor_basal_rev` of the target ITP,
- Modify the model’s dynamics so that
- The model state converges to a new attractor within the altered dynamics,
- And when, from that attractor, an input-configuration-dependent attractor transition occurs, leading to an iCA,
- The average phenotype states of that iCA under `IC_basal` should closely resemble those of `attractor_basal_irrev`,
- And the average phenotype states of that iCA under `IC_transition` should closely resemble those of `attractor_transition`.

The detailed procedure of reversibility-inducing control is as follows:
- Fix the control targets to their corresponding assigned states,
- Allow the model state to converge to a new attractor within the modified dynamics,
- Then identify the iCAs that can be reached from this attractor via input-configuration-dependent attractor transitions.

Each control configuration is evaluated by a 'desired phenotype score', which quantifies how well it achieves the goals of reversibility-inducing control.<br>
ROOT assumes that, among all control configurations with the same number of control targets, those with the highest desired phenotype scores are most likely to be included among the effective control candidates.<br>
Based on this assumption, ROOT calculates the desired phenotype score for each control configuration and uses these scores to identify the most suitable configurations for reversibility-inducing control.

### Setting Phenotype Nodes
To calculate the desired phenotype score, the phenotype of the model must be defined.<br>
To do this, phenotype nodes—the nodes used to define the model’s phenotype—must be specified.

In [38]:
###############################################################################
######       In this cell, the user must input appropriate values.       ######
###############################################################################

phenotype_nodes = ["Pheno1","Pheno2"]

The following cell shows a table that lists each previously identified control candidate along with its desired phenotype score and its rank within the corresponding ITP. For each ITP, the control configurations with rank 1 are those proposed by ROOT as the most suitable for reversibility-inducing control.

In [39]:
def compute_competition_ranks(values):
    indexed = list(enumerate(values))
    sorted_indexed = sorted(indexed, key=lambda x: -x[1])

    ranks = [0] * len(values)
    rank = 1
    for i, (idx, val) in enumerate(sorted_indexed):
        if i > 0 and val != sorted_indexed[i - 1][1]:
            rank = i + 1
        ranks[idx] = rank

    return ranks

control_configurations_for_all_analyzed_ITPs = []
for control_configurations in ITPindex_controlconfigurations_map.values():
    for control_configuration in control_configurations:
        if control_configuration not in control_configurations_for_all_analyzed_ITPs:
            control_configurations_for_all_analyzed_ITPs.append(control_configuration)

index_of_df = []
control_configurations_in_df = []
control_configuration_is_for_the_ITP = []
desired_phenotype_score_in_df = []
ranks_in_the_ITP = []

for ITP_index, control_configurations_for_the_ITP in ITPindex_controlconfigurations_map.items():
    ITP_object = ITP_index_map[ITP_index]
    for control_configuration in control_configurations_for_all_analyzed_ITPs:
        index_of_df.append(ITP_index)
        control_configurations_in_df.append(control_configuration)
        if control_configuration in control_configurations_for_the_ITP:
            control_configuration_is_for_the_ITP.append(True)
        else:
            control_configuration_is_for_the_ITP.append(False)
        desired_phenotype_score_for_control_configuration = ITP_object.get_desired_phenotype_score_for_reversibility_inducing_control(control_configuration, phenotype_nodes)
        desired_phenotype_score_in_df.append(desired_phenotype_score_for_control_configuration)
ranks_in_the_ITP = compute_competition_ranks(desired_phenotype_score_in_df)

df_reversibility_inducing_control = pd.DataFrame({"control configuration":control_configurations_in_df,
                                     "control configuration is candidate control for the ITP":control_configuration_is_for_the_ITP,
                                       "desired phenotype score":desired_phenotype_score_in_df,
                                       "rank in the ITP":ranks_in_the_ITP},
                                      index=index_of_df)
df_reversibility_inducing_control.index.name = 'ITP index'
df_reversibility_inducing_control = df_reversibility_inducing_control.sort_values("rank in the ITP")

print("control configurations and their desired phenotype score per ITP")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_reversibility_inducing_control)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

control configurations and their desired phenotype score per ITP


,control configuration,control configuration is candidate control for the ITP,desired phenotype score,rank in the ITP
ITP index,,,,
1,{'X06': 0},True,1.0,1
1,{'X08': 0},True,1.0,1


The following table shows the average desired phenotype score for each control configuration, the frequency with which each was selected as a candidate control across ITPs, and their rank based on the average desired phenotype socre. When attempting to apply reversibility-inducing control across multiple ITPs simultaneously, the control configurations ranked 1 are those recommended by ROOT as the most suitable.

In [ ]:
desired_phenotype_score_averaged = []
ratios_of_being_candidate_control = []
ranks_averaged = []
for control_configuration in control_configurations_for_all_analyzed_ITPs:
    sub_df_for_the_control_configuration = df_reversibility_inducing_control[df_reversibility_inducing_control["control configuration"] == control_configuration]
    
    desired_phenotype_score_averaged_of_the_control_configuration = sub_df_for_the_control_configuration["desired phenotype score"].mean()
    desired_phenotype_score_averaged.append(desired_phenotype_score_averaged_of_the_control_configuration)
    ratio_of_being_condidate_for_the_control_configuration = sub_df_for_the_control_configuration["control configuration is candidate control for the ITP"].mean()
    ratios_of_being_candidate_control.append(ratio_of_being_condidate_for_the_control_configuration)
    ranks_averaged = compute_competition_ranks(desired_phenotype_score_averaged)

df_reversibility_inducing_control_statistic = pd.DataFrame({"average of desired phenotype score":desired_phenotype_score_averaged,
                                     "ratio of being candidate control":ratios_of_being_candidate_control,
                                     "average rank":ranks_averaged},
                                      index=control_configurations_for_all_analyzed_ITPs)
df_reversibility_inducing_control_statistic.index.name = 'control configuration'
df_reversibility_inducing_control_statistic = df_reversibility_inducing_control_statistic.sort_values("average rank")

print("Summary of averaged desired phenotype score, ratio of being candidate control, and average rank")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_reversibility_inducing_control_statistic)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Summary of averaged desired phenotype score, ratio of being candidate control, and average rank


,average of desired phenotype score,ratio of being candidate control,average rank
control configuration,,,
{'X06': 0},1.0,1.0,1.0
{'X08': 0},1.0,1.0,1.0
